# Seurat object  
## Load required packages


In [ ]:
library(Seurat)
library(tidyverse)
library(ggplot2)



## Load seurat object (NORM)


In [ ]:
line <- params$line



In [ ]:
seurat_obj <- readRDS(file = paste0("./results/rds/samples/", line, "_FindNeighbors.rds"))

seurat_obj
table(seurat_obj$orig.ident)



## <span style="color:#296d98;"> Parameters </span>  


In [ ]:
res = as.double(params$res)
res


# Defining cell clusters  
## Optimize resolution  


In [ ]:
seurat_obj <- FindClusters(seurat_obj, resolution = res)
levels(seurat_obj$seurat_clusters)



## Plot clusters


In [ ]:
DimPlot(seurat_obj, reduction = "pca", label = TRUE, group.by = "seurat_clusters") + ggtitle("PC1 vs PC2 with Clusters")



In [ ]:
DimPlot(seurat_obj, reduction = "umap", group.by = "seurat_clusters")
DimPlot(seurat_obj, reduction = "umap", group.by = "seurat_clusters", label.size = 4, label = TRUE)+ NoLegend()


In [ ]:
DimPlot(seurat_obj, reduction = "tsne", group.by = "seurat_clusters")
DimPlot(seurat_obj, reduction = "tsne", group.by = "seurat_clusters", label.size = 4, label = TRUE)+ NoLegend()



### Split by condition


In [ ]:
DimPlot(seurat_obj, reduction = "umap", group.by="seurat_clusters", split.by = "condition")
DimPlot(seurat_obj, reduction = "umap", group.by = "seurat_clusters", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(seurat_obj, reduction = "umap", group.by="seurat_clusters", split.by = "condition")+ NoLegend()


In [ ]:
DimPlot(seurat_obj, reduction = "tsne", group.by="seurat_clusters", split.by = "condition")
DimPlot(seurat_obj, reduction = "tsne", group.by = "seurat_clusters", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(seurat_obj, reduction = "tsne", group.by="seurat_clusters", split.by = "condition")+ NoLegend()


## QC metrics  
Now that we have our clusters, we can look to see if they are being influenced by any of the QC metrics.  

### Nb of reads  


In [ ]:
VlnPlot(seurat_obj, features="nCount_RNA", group.by = "seurat_clusters", pt.size = 0)
RidgePlot(seurat_obj, features = "nCount_RNA", group.by = "seurat_clusters")



### Nb of genes    


In [ ]:
VlnPlot(seurat_obj, features="nFeature_RNA", group.by = "seurat_clusters", pt.size = 0)
RidgePlot(seurat_obj, features = "nFeature_RNA", group.by = "seurat_clusters")



### MT genes  


In [ ]:
VlnPlot(seurat_obj, features = "percent.mt", group.by = "seurat_clusters", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.mt", group.by = "seurat_clusters")



### Ribosomal genes  


In [ ]:
VlnPlot(seurat_obj, features = "percent.ribosomal", group.by = "seurat_clusters", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.ribosomal", group.by = "seurat_clusters")



### Largest gene  


In [ ]:
VlnPlot(seurat_obj, features = "percent.largest_gene", group.by = "seurat_clusters", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.largest_gene", group.by = "seurat_clusters")


In [ ]:
# which largest gene
seurat_obj[[]] %>%
  group_by(seurat_clusters, largest_gene) %>%
  count() %>%
  arrange(desc(n)) %>%
  group_by(seurat_clusters) %>%
  slice(1:2) %>%
  ungroup() %>%
  arrange(seurat_clusters, desc(n))



### Cell cycle  


In [ ]:
seurat_obj@meta.data %>%
  group_by(seurat_clusters,Phase) %>%
  count() %>%
  group_by(seurat_clusters) %>%
  mutate(percent=100*n/sum(n)) %>%
  ungroup() %>%
  ggplot(aes(x=seurat_clusters,y=percent, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' )) +
  geom_col() +
  ggtitle("Percentage of cell cycle phases per sample")


In [ ]:
tibble(
  cluster = seurat_obj$seurat_clusters,
  Phase = seurat_obj$Phase,
) %>%
  group_by(cluster,Phase) %>%
  count() %>%
  group_by(cluster) %>%
  mutate(
    percent=(100*n)/sum(n)
  ) %>%
  ungroup() %>%
  mutate(
    cluster=paste("Cluster",cluster)
  ) %>%
  ggplot(aes(x="",y=percent, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' )) +
  geom_col(width=1) +
  coord_polar("y", start=0) +
  facet_wrap(vars(cluster)) +  
  theme(axis.text.x=element_blank()) +
  xlab(NULL) +
  ylab(NULL)



### Orig ident


In [ ]:
tibble(
  cluster = seurat_obj$seurat_clusters,
  orig = seurat_obj$orig.ident,
) %>%
  group_by(cluster,orig) %>%
  count() %>%
  group_by(cluster) %>%
  mutate(
    percent=(100*n)/sum(n)
  ) %>%
  ungroup() %>%
  mutate(
    cluster=paste("Cluster",cluster)
  ) %>%
  ggplot(aes(x="",y=percent, fill=orig)) +
  geom_col(width=1) +
  coord_polar("y", start=0) +
  facet_wrap(vars(cluster)) +  
  theme(axis.text.x=element_blank()) +
  xlab(NULL) +
  ylab(NULL)


# Finding markers for each cluster  
Identify genes whose expression defines each cluster which has been identified.

* **The Wilcox rank sum test**: This identifies genes which are differentially regulated between two groups of cells. It is a non-parametric test which makes very few assumptions about the behaviour of the data and just looks for genes which have expression which is consistently ranked more highly in one group of cells compared to another.  

* **The ROC test**: This is a measure of how specifically a gene can predict membership of two groups. It gives a value between 0.5 (no predictive value) and 1 (perfectly predictive on its own) to say how useful each gene is at predicting. Again this is a non-parametric test which just cares about the ranked expression measures for each gene.


## Biomarkers  

As we are working on cells from 2 different conditions (*prolif* or *diff*), we won't use `FindAllMarkers()` but `FindConservedMarkers()`.   
It will separate the cells between *prolif* and *diff* conditions and then find DE genes betweeen clusters. Hence we won't have markers genes kept because of the difference in conditions but due to the difference in clusters.

### Clusters markers  
`PrepSCTFindMarkers()` ensures that the fixed value is set properly before DE analysis.
*(use this function for a merged object with multiple SCT models)*


In [ ]:
seurat_obj <- PrepSCTFindMarkers(seurat_obj)

lapply(
  levels(seurat_obj$seurat_clusters),
  function(x)FindConservedMarkers(seurat_obj, assay = "SCT", ident.1 = x, grouping.var='condition', verbose=FALSE)
) -> cluster.markers

# This adds the cluster number to the results of FindConservedMarkers
sapply(0:(length(cluster.markers)-1),function(x) {
  cluster.markers[[x+1]]$gene <<- rownames(cluster.markers[[x+1]])
  cluster.markers[[x+1]]$cluster <<- x
})



* `DoHeatmap` generates an expression heatmap for given cells and features. In this case, we are plotting the top 20 markers (or all markers if less than 20) for each cluster.   


In [ ]:
cluster.markers <- do.call(rbind.data.frame, cluster.markers)
dim(cluster.markers)


In [ ]:
prolif <- cluster.markers[which( cluster.markers$diff_p_val == cluster.markers$max_pval), ]
diff <- cluster.markers[which( cluster.markers$prolif_p_val == cluster.markers$max_pval), ]
dim(prolif)
dim(diff)


In [ ]:
prolif <- prolif[, c('prolif_p_val', 'prolif_avg_log2FC', 'prolif_p_val_adj', 'gene', 'cluster' )]
head(prolif)


In [ ]:
diff <- diff[, c('diff_p_val', 'diff_avg_log2FC', 'diff_p_val_adj', 'gene', 'cluster' )]
head(diff)


In [ ]:
diff_markers <- diff[order(diff$diff_p_val_adj, decreasing = FALSE),]
prolif_markers <- prolif[order(prolif$prolif_p_val_adj, decreasing = FALSE),]


In [ ]:
diff_markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = diff_avg_log2FC) -> top10
DoHeatmap(metabo_obj, features = top10$gene) + NoLegend()

prolif_markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = prolif_avg_log2FC) -> top10
DoHeatmap(metabo_obj, features = top10$gene) + NoLegend()



### Save markers lists


In [ ]:
saveRDS(cluster.markers, file = paste0( "./results/html/",line, "/", line, "_cluster.markers.rds"))




# Save seurat object  


In [ ]:
saveRDS(seurat_obj, file = paste0("./results/rds/samples/", line, "_Markers.rds"))

